# Classifying trialwise CorrectGo and NoGo trials

There are a number of steps to this. Hopefully we can recycle previous code and be up fairly quickly!

1. Load beta data. Ideally this process should include a cache into a pure python object so we don't have to reload it each time.
2. Preprocess the data.
3. Do cross-validated training and testing. Ideally an inner loop to select best parameters, an outer loop to get cross-validated performance, and final training over all the data to get an image. The inner loop can be probably be handled within the package we use probably.

In [1]:
import socket
import yaml
hostname=socket.gethostname()
hostname='zzz'
with open('sst_config.yml', "r") as f:
    test_config= yaml.safe_load(f)#[hostname]

In [2]:
import sys
import os
import pandas as pd
import numpy as np



sys.path.append(os.path.abspath("../../ml/"))
from apply_loocv_and_save import load_and_preprocess
from dev_utils import read_yaml_for_host
from dev_utils import get_2DX_from_4DX
import warnings


config_data = read_yaml_for_host("sst_config.yml")



python initialized for apply_loocv_and_save
cpus available; cpus to use:
10 9
10


In [3]:
import multiprocessing
import math
import nibabel as nib
import nilearn as nl
from nilearn.decoding import DecoderRegressor,Decoder
from sklearn.model_selection import KFold,GroupKFold,LeaveOneOut
cpus_available = multiprocessing.cpu_count()

cpus_to_use = min(cpus_available-1,math.floor(0.9*cpus_available))
print(cpus_to_use)

9


In [4]:
from dev_wtp_io_utils import cv_train_test_sets, asizeof_fmt
from nilearn.decoding import DecoderRegressor,Decoder

In [5]:
nonbids_data_path = config_data['nonbids_data_path']
ml_data_folderpath = nonbids_data_path + "fMRI/ml"


## Set up the paradigm

In [6]:

def trialtype_resp_trans_func(X):
    return(X.trial_type)


## Loading beta data

beta data is generally written in `load_multisubject_brain_data_sst_w1.ipynb`.

We just have to load it.

In [7]:
#brain_data_filepath = ml_data_folderpath + '/SST/Brain_Data_betaseries_30subs_correct_cond_pfc.pkl'
brain_data_filepath = ml_data_folderpath + '/SST/Brain_Data_betaseries_6subs_correct_cond.pkl'
warnings.warn("not sure if this file holds up--it was created in 2021; need to see if it's still valid")
train_test_markers_filepath = ml_data_folderpath + "/train_test_markers_20220818T144138.csv"

/var/folders/cj/4mb6t1f906j397tj71pxfxz00000gn/T/ipykernel_29428/2753699166.py:3: UserWarning: not sure if this file holds up--it was created in 2021; need to see if it's still valid
  warnings.warn("not sure if this file holds up--it was created in 2021; need to see if it's still valid")


In [8]:


all_subjects = load_and_preprocess(
    brain_data_filepath,
    train_test_markers_filepath,
    subjs_to_use = None,
    response_transform_func = trialtype_resp_trans_func,
    clean=None)

warnings.warn("the data hasn't been cleaned at any point. the fMRIPrep cleaning pipeline has been applied; nothing else has been.")


checked for intersection and no intersection between the brain data and the subjects was found.
there were 6 subjects overlapping between the subjects marked for train data and the training dump file itself.


/Users/benjaminsmith/opt/anaconda3/envs/neuralsignature/lib/python3.10/site-packages/nilearn/input_data/__init__.py:27: FutureWarning: The import path 'nilearn.input_data' is deprecated in version 0.9. Importing from 'nilearn.input_data' will be possible at least until release 0.13.0. Please import from 'nilearn.maskers' instead.
  warnings.warn(message, FutureWarning)


test_train_set: 62918
brain_data_filepath: 168
pkl_file: 168
train_test_markers_filepath: 158
response_transform_func: 144
sys: 72
Brain_Data_allsubs: 48
clean: 16
subjs_to_use: 16


/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/fMRI/ml/apply_loocv_and_save.py:217: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Brain_Data_allsubs.Y[Brain_Data_allsubs.Y=='NULL']=None


601
601
cleaning memory


/var/folders/cj/4mb6t1f906j397tj71pxfxz00000gn/T/ipykernel_29428/1474138623.py:8: UserWarning: the data hasn't been cleaned at any point. the fMRIPrep cleaning pipeline has been applied; nothing else has been.
  warnings.warn("the data hasn't been cleaned at any point. the fMRIPrep cleaning pipeline has been applied; nothing else has been.")


In [9]:
from sklearn.feature_selection import SelectPercentile,f_classif

#convert the y array to an integer array representing the string values of the y array
all_subjects['y_cat'] = all_subjects['y'].astype('category')
all_subjects['y_int']=all_subjects['y_cat'].cat.codes

# #get the X fdata
# x_fdata = all_subjects['X'].get_fdata()
# print(x_fdata.shape)
# #flatten first 3 dims of X fdata to a single dim
# x_fdata_1d = x_fdata.reshape([np.prod(x_fdata.shape[0:3]), x_fdata.shape[3]])
# print(x_fdata_1d.shape)



# #SelectPercentile(f_classif,percentile=5).fit_transform(,all_subjects['y_int'])

In [10]:
mask_nifti = nib.load(ml_data_folderpath + '/prefrontal_cortex.nii.gz')

In [11]:
num_subjs = 6
if num_subjs<len(all_subjects['metadata']['subject'].unique()):
    #cut down to just 9 subjects
    selected_subjs={}
    print(all_subjects.keys())

    #select subjs
    subjs = all_subjects['metadata']['subject'].unique()
    subjs.sort()
    selected_sub_ids=subjs[0:num_subjs]
    #get index of selected subjs
    selected_subjs_idx_bool = all_subjects['metadata']['subject'].isin(selected_sub_ids)
    #get list of True indices
    selected_subjs_idx = selected_subjs_idx_bool[selected_subjs_idx_bool].index.tolist()

    subjs = all_subjects['metadata']['subject'].unique()
    selected_subjs['X'] = nib.funcs.concat_images([all_subjects['X'].slicer[...,s] for s in selected_subjs_idx])
    selected_subjs['y'] = all_subjects['y'][selected_subjs_idx_bool]

    selected_subjs['y'] = all_subjects['y'][selected_subjs_idx_bool]
    selected_subjs['groups'] = all_subjects['groups'][selected_subjs_idx_bool]
    selected_subjs['metadata'] = all_subjects['metadata'][selected_subjs_idx_bool]
else:
    selected_subjs = all_subjects

In [12]:
# get the PFC mask
mask_nifti = nib.load(ml_data_folderpath + '/prefrontal_cortex.nii.gz')

In [13]:
#convert the y array to an integer array representing the string values of the y array
selected_subjs['y_cat'] = selected_subjs['y'].astype('category')
selected_subjs['y_int']=selected_subjs['y_cat'].cat.codes

In [14]:
X=selected_subjs['X']

In [24]:


def get_2DX_from_4DX(nifti_X):
    #rotate my array so that the last dimension is first
    print(nifti_X.shape)
    nifti_X=np.moveaxis(nifti_X,-1,0)
    print(nifti_X.shape)
    pre_reshaped_shape = nifti_X.shape
    #now flatten dims 2-4 into a single dimension
    arr2d=myarr.reshape([nifti_X.shape[0],np.prod(nifti_X.shape[1:4])])
    print("this is the form of the data that the decoder wants, (n_samples, n_features)")
    print(arr2d.shape)
    return(arr2d)


def get_4DX_from_2DX(arr2d,pre_reshaped_shape):
    #now undo the above operations to get the original matrix
    print(arr2d.shape)
    arr4d=arr2d.reshape(pre_reshaped_shape)
    print(myarr_rev.shape)
    arr4d=np.moveaxis(arr4d,0,-1)
    print(arr4d.shape)
    return(arr4d)

In [22]:
#from dev_utils import get_2DX_from_4DX
X_2d=get_2DX_from_4DX(X.get_fdata())

(97, 115, 97, 601)


NameError: name 'np' is not defined

In [21]:
X_2d.shape

NameError: name 'X_2d' is not defined

In [18]:

X.shape
print([np.prod(X.shape[0:3]),X.shape[3]])
print(X.get_fdata().shape)
#collapse the first 3 dims of X fdata to a single dim
X_reshaped = np.reshape(X.get_fdata(),[np.prod(X.shape[0:3]),X.shape[3]]).shape

[1082035, 601]
(97, 115, 97, 601)


KeyboardInterrupt: 

In [26]:
#reshape the X data to a 2D array
def preprocess_func(X):
    X_2d = np.reshape(X.get_fdata(),[np.prod(X.shape[0:3]),X.shape[3]])
    X_nifti = nl.image.new_img_like(X,X_2d)
    return(X_nifti)

#apply the preprocessing function to the X data
selected_subjs['X2'] = preprocess_func(selected_subjs['X'])

# Training

I'm going to start with `cv_train_test_sets` and see how that goes. It sems likely it'll have to be re-written somewhat, but it might be a good starting point.

I think I need to run this without all the extra scaffolding--just tresting the Decoder on the data until I get something sensible. At the very least we need to know the Decoder object is handling balanced classes correctly.

In [15]:
# use add PFC mask.ipynb to figure out how to get a PFC mask onto this data.

In [16]:
transformer = f_classif

In [17]:
from EstimatorWithPreprocessing import EstimatorWithPreprocessor
#import SVC
from sklearn.svm import SVC, LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import f_classif, SequentialFeatureSelector, SelectKBest


ewp = EstimatorWithPreprocessor(
    estimator=LinearSVC(penalty='l2',max_iter=1e4),
    preprocessor= 
)



In [25]:
k_fit = SelectKBest(f_classif, k=1000).fit(selected_subjs['X'],y=np.array(selected_subjs['y_int']))

ValueError: Expected 2D array, got scalar array instead:
array=<class 'nibabel.nifti1.Nifti1Image'>
data shape (97, 115, 97, 601)
affine: 
[[   2.    0.    0.  -96.]
 [   0.    2.    0. -132.]
 [   0.    0.    2.  -78.]
 [   0.    0.    0.    1.]]
metadata:
<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b''
dim_info        : 0
dim             : [  4  97 115  97 601   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : float32
bitpix          : 32
slice_start     : 0
pixdim          : [1. 2. 2. 2. 1. 1. 1. 1.]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 0
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : b''
aux

In [18]:
#dec_main = Decoder(standardize=True,cv=GroupKFold(3),scoring='roc_auc',n_jobs=cpus_to_use,mask=mask_nifti)
cv_results = cv_train_test_sets(
    trainset_X = selected_subjs['X'],
    trainset_y = selected_subjs['y_int'],
    trainset_groups = selected_subjs['metadata']['subject'],
    decoders = [ewp],
    cv=KFold(n_splits=3) # we use KFold, not GroupKfold, because it's splitting on Group anyway
    )

Groups are the same.
fold 1 of 3
In order to test on a training group of 4 items, holding out the following subjects:['DEV005' 'DEV010']. prepping fold data.... fitting.... 3.3 GiB. trying decoder 1 of 1. 

ValueError: Expected 2D array, got scalar array instead:
array=<class 'nibabel.nifti1.Nifti1Image'>
data shape (97, 115, 97, 409)
affine: 
[[   2.    0.    0.  -96.]
 [   0.    2.    0. -132.]
 [   0.    0.    2.  -78.]
 [   0.    0.    0.    1.]]
metadata:
<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b''
dim_info        : 0
dim             : [  4  97 115  97 409   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : float32
bitpix          : 32
slice_start     : 0
pixdim          : [1. 2. 2. 2. 1. 1. 1. 1.]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 0
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : b''
aux

In [ ]:
cv_results_dict = {
    'test_scores':cv_results[0],
    'results':cv_results[1],
    'results_by_trainset_item':cv_results[2],
}

In [ ]:
cv_results_dict['results_by_trainset_item']

The following is derived from 3-fold cross-validation, and should indicate train/test performance classifying trials across subjects.

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
#get precision and recall
print(precision_recall_fscore_support(
    cv_results_dict['results_by_trainset_item']['y'],
cv_results_dict['results_by_trainset_item']['y_pred'].astype(int),average='macro'))

#get roc_auc
from sklearn.metrics import roc_auc_score
print(roc_auc_score(
    cv_results_dict['results_by_trainset_item']['y'],
    cv_results_dict['results_by_trainset_item']['y_pred']))